# Example/ concept notebook for catalog search sky viewer example

The use case itself is detailed [here](sky-viewer-catalog-search.md).  This notebook gives an idea of how this use case would work.  Some/all of the code may not work, but the goal is for the notebook (or a very similar notebook) to work once the development work has been completed.

* Note: throughout this example, the assumption is that Roman catalog-related functionality all lives in `astroquery.mast.Catalogs` this does not *need* to be the case - some of the functionality could live in some other helper package.  But for the sake of example we will assume all-in-astroquery.

In [ ]:
# this cell would need to have the annotation that it is a parameter cell for papermill - see more info below
distancempc = 1
targetradeg = 123.4
targetdecdeg = 5.6

In [ ]:
from astroquery.mast import Catalogs
from astroquery.mast import Viewers
from astropy import units as υ
from astropy.coordinates import Distance

from matplotlib import pyplot as plt

In [ ]:
Catalog.get_schema('roman-sources')


The above should either pop-up a Jupyterlab window, or a link to a web page with the relevant catalog's schema.  The user then sees that there are columns named after ther relevant filters - e.g. 'F062_mag' and 'F106_mag'.  That shows the user what magnitudes they need to define their color-magnitude box.  

In [ ]:
# the user would specify this based on outside knowledge of where RGB stars are in these filters
filter1 = 'F062'
filter2 = 'F106'
rgb_min_absmag = XXX 
rgb_max_absmag = XXX 
rgb_min_color = XXX 
rgb_min_color = XXX 

targetfield = SkyCoord(targetradeg*u.deg, targetdecdeg*u.deg, frame='icrs')
fieldsize = 1*u.deg

distance_moduli = (distancempc*u.Mpc).distmod

min_mag = rgb_min_absmag + distance_moduli
max_mag = rgb_max_absmag + distance_moduli

## 1. The "query_criteria" way

This is the way to get the catalogs using something comparable to the `query_criteria` function currently in astroquery.mast.Catalog.  Note that it requires that there already be a field in the database for the color, despite that being something directly derived from the two magnitudes

In [ ]:
catalog = Catalog.query_criteria(targetfield, radius=fieldsize, 
                    F106_mag=('gte', min_mag),
                    F106_mag=('lte', max_mag), 
                    F062F106_color=('gte', rgb_min_color),
                    F062F106_color=('lte', rgb_max_color),
                    catalog='roman-sources')

## 2. The ADQL way

The user could also use an ADQL query if they understand ADQL sufficiently, run that some  other way, and load those resulting tables explicitly here.  (Code not shown, but it should yield the same list of catalogs as the other cases)

## 3. The "dynamic" way (stretch goal)

This is an alternative approach that uses the stretch goal formulation described in [this use case](database-access-dynamic.md).  It would transparently map the masking operations to the databse instead of requirng the user to do simplistic queries against

In [ ]:
deferred = Catalog.query_criteria_deferred(targetfield, radius=fieldsize, catalog='roman-sources')
mag1 = deferred['F062_mag']
mag2 = deferred['F106_mag']
color = mag2 - mag1
query = (min_mags < mag2) & (mag2 < max_mags) & (rgb_min_color < color) & (color < rgb_max_color)

catalog = query.execute()

Regardless of 1,2,3 above, the user now has a set of catalog of all the RGB candiate objects. The next step is to produce some general plots color-magnitude diagrams for these.  In a real practical case there'd probably need to be some code to clean up the plots and make sure outliers are removed, but we won't worry about that right now because it's a detail that depends on the data at some level.

In [ ]:
plt.figure()
plt.scatter(catalog['F062_mag'], catalog['F106_mag'] - catalog['F062_mag'])

plt.plot(something_informational_like_an_isocrone)

plt.figure()
plt.scatter(catalog['coord'].ra.deg, catalog['coord'].dec.deg)  # this assumes the catalog automatically comes with a SkyCoord mixin column

In [ ]:
skyviewer = Viewers.show_sky_viewer(targetfield)
skyviewer.show_in_viewer(catalog)  # this would automatically recognize that 'coord' is the SkyCoord column and use that for plotting

# Steps to run in a separate notebook

In this use case, the user wants to produce a bunch of copies of the above notebook for their collaborators to view.  A tool called `papermill` already exists to do this, in which case the user would need to use a different notebook to actually *run* papermill, with the above as a template.  The content below is for the *runner* notebook

In [ ]:
from papermill.execute import execute_notebooks

In [ ]:
distances = [1, 2,3 ]*u.Mpc
coordinates = ... a sky coord object with all the various targets that they get from somewhere else...

In [ ]:
for i, targetcoo in enumerate(coordinates):
    for d in distancees:
        dmpc = d.to(u.Mpc).value
        radeg = targetcoo.ra.deg
        decdeg = targetcoo.dec.deg

        execute_notebooks('sky-viewer-catalog-search.ipynb', f'/path/to/relevant/roman/group/target{i}_distance{dmpc}.ipynb',
                          parameters={'distancempc': dmpc, 'targetradeg': radeg, 'targetdecdeg': decdeg},
                          prepare_only=True)

The result is a bunch of notebooks ready to be excuted by the team, and the team lead distributes them to the team for visual inspection.